In [2]:
import pandas as pd, numpy as np

In [2]:
# For laptop
klstVedurPath = 'D:\Skóli\lokaverkefni_vel\data\Vedurstofa\stripped_klst.feather'
mergedPath = 'D:\Skóli\lokaverkefni_vel\data\merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather'

In [3]:
# For PC
klstVedurPath = 'E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/combined_klst.feather'
mergedPath = 'E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather'

In [81]:
df_vedur_klst = pd.read_feather(klstVedurPath)
df_merged = pd.read_feather(mergedPath)

In [82]:
df_vedur_klst.shape

(30220136, 11)

In [84]:
df_temp = df_vedur_klst.drop(index = 26952999)

df_temp.sort_values(by=['timi'])

,timi,stod,f,fx,fg,d,dsdev,longitude,latitude,X,Y
25735444,1990-02-19 07:00:00,6300,5.3,7.0,8.3,45.0,14.1,-20.9707,63.9355,403355.367828,382834.566367
25735445,1990-02-19 08:00:00,6300,5.9,5.9,8.0,46.0,10.4,-20.9707,63.9355,403355.367828,382834.566367
25735446,1990-02-19 09:00:00,6300,3.1,6.9,10.0,49.0,14.3,-20.9707,63.9355,403355.367828,382834.566367
25735447,1990-02-19 10:00:00,6300,6.4,6.4,8.7,35.0,9.7,-20.9707,63.9355,403355.367828,382834.566367
25735448,1990-02-19 11:00:00,6300,6.2,7.1,10.3,41.0,13.1,-20.9707,63.9355,403355.367828,382834.566367
...,...,...,...,...,...,...,...,...,...,...,...
25773064,2023-10-05 09:00:00,6300,4.7,4.7,7.7,53.0,15.9,-20.9707,63.9355,403355.367828,382834.566367
29830795,2023-10-05 09:00:00,7001,9.4,9.4,11.7,60.0,NaN,-22.2496,63.9207,340596.727199,383775.944895
19674959,2023-10-05 09:00:00,4921,4.5,4.5,6.1,91.0,7.6,-16.1441,66.5114,627060.220015,671402.647156
14687304,2023-10-05 09:00:00,3696,1.0,1.0,2.3,1.0,19.1,-17.3281,66.0418,575785.171221,617159.740022


In [85]:
df_vedur_klst = df_temp

In [86]:
df_vedur_klst.timi = pd.to_datetime(df_vedur_klst.timi)
df_merged.time = pd.to_datetime(df_merged.time)

In [87]:
# Rounds to next hour so as to be able to directly compare with the vedur klst file
def next_hour(time):
    return time.ceil('h')

In [88]:
df_merged['timi'] = df_merged.time.apply(next_hour)

In [89]:
df_merged.columns

Index(['X', 'Y', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250',
       'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'stod',
       'f', 'fg', 'fsdev', 'd', 'dsdev', 'longitude', 'latitude', 'Ri_01',
       'Ri_12', 'N_01', 'N_12', 'station_elevation', 'landscape_points',
       'elevations', 'timi'],
      dtype='object')

In [90]:
df_vedur_klst.columns

Index(['timi', 'stod', 'f', 'fx', 'fg', 'd', 'dsdev', 'longitude', 'latitude',
       'X', 'Y'],
      dtype='object')

In [91]:
df_vedur_klst.stod = df_vedur_klst.stod.astype(np.int64)
df_vedur_klst.fg = pd.to_numeric(df_vedur_klst.fg, errors = 'coerce', downcast='float')

In [93]:
df_merged_vedur = pd.merge(df_merged, df_vedur_klst, on = ['stod', 'timi'], how = 'inner', suffixes=('_merged', '_vedur'))

In [94]:
df_merged_vedur

,X_merged,Y_merged,time,ws_15,ws_250,ws_500,wd_15,wd_250,wd_500,p_15,...,timi,f_vedur,fx,fg_vedur,d_vedur,dsdev_vedur,longitude_vedur,latitude_vedur,X_vedur,Y_vedur
0,419760.4353,311410.4631,2020-02-14 09:40:00,19.6041,23.1354,23.5411,97.9485,99.3434,99.9549,98956.6580,...,2020-02-14 10:00:00,23.6,27.5,35.599998,78.0,7.3,-20.5995,63.2993,419760.435262,311410.463057
1,419760.4353,311410.4631,2019-02-20 01:00:00,14.4611,16.8599,17.1642,145.3825,146.3173,146.5913,100351.7109,...,2019-02-20 01:00:00,25.7,25.8,34.200001,83.0,14.7,-20.5995,63.2993,419760.435262,311410.463057
2,419760.4353,311410.4631,2019-02-17 00:40:00,8.5471,9.1812,9.1686,62.6103,65.3626,79.3139,100649.9705,...,2019-02-17 01:00:00,24.2,25.7,34.299999,75.0,15.8,-20.5995,63.2993,419760.435262,311410.463057
3,419760.4353,311410.4631,2019-02-05 23:40:00,10.3157,11.1665,11.1894,88.1111,89.3584,90.2917,101001.3064,...,2019-02-06 00:00:00,24.0,25.2,33.299999,86.0,10.4,-20.5995,63.2993,419760.435262,311410.463057
4,419760.4353,311410.4631,2018-12-17 19:30:00,11.2918,17.3783,17.6068,116.0218,129.5856,136.2121,99682.9219,...,2018-12-17 20:00:00,23.6,27.7,36.599998,76.0,9.5,-20.5995,63.2993,419760.435262,311410.463057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12702,543704.3472,672486.1026,2014-12-10 12:40:00,10.6554,11.5584,11.0643,6.4581,8.6411,12.4018,96755.1484,...,2014-12-10 13:00:00,23.3,27.3,35.500000,23.0,19.3,-18.0167,66.5438,543704.347236,672486.102600
12703,543704.3472,672486.1026,2015-12-08 05:00:00,25.4698,31.2753,33.5225,97.3362,98.8644,101.6907,99002.3281,...,2015-12-08 05:00:00,26.6,26.6,33.400002,85.0,6.0,-18.0167,66.5438,543704.347236,672486.102600
12704,543704.3472,672486.1026,2019-12-11 08:10:00,19.3720,22.8016,23.2493,41.1882,44.0706,46.2614,95774.5937,...,2019-12-11 09:00:00,22.1,25.9,33.200001,359.0,21.1,-18.0167,66.5438,543704.347236,672486.102600
12705,543704.3472,672486.1026,2020-02-14 14:10:00,23.5278,28.6951,29.9320,93.5311,95.0293,96.2047,97827.6385,...,2020-02-14 15:00:00,23.3,25.0,35.799999,81.0,8.3,-18.0167,66.5438,543704.347236,672486.102600


In [102]:
df_final = df_merged_vedur[abs(df_merged_vedur.f_merged - df_merged_vedur.fx) <= .1]

df_final.shape

(10365, 40)

In [104]:
df_final.columns

Index(['X_merged', 'Y_merged', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15',
       'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500',
       'stod', 'f_merged', 'fg_merged', 'fsdev', 'd_merged', 'dsdev_merged',
       'longitude_merged', 'latitude_merged', 'Ri_01', 'Ri_12', 'N_01', 'N_12',
       'station_elevation', 'landscape_points', 'elevations', 'timi',
       'f_vedur', 'fx', 'fg_vedur', 'd_vedur', 'dsdev_vedur',
       'longitude_vedur', 'latitude_vedur', 'X_vedur', 'Y_vedur'],
      dtype='object')

In [111]:
df_toWrite = df_final[['X_merged', 'Y_merged', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15',
       'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500',
       'stod', 'f_merged', 'fg_merged', 'd_merged', 'longitude_merged', 'latitude_merged', 'Ri_01', 'Ri_12', 'N_01', 'N_12',
       'station_elevation', 'landscape_points', 'elevations']]

df_toWrite.columns

Index(['X_merged', 'Y_merged', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15',
       'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500',
       'stod', 'f_merged', 'fg_merged', 'd_merged', 'longitude_merged',
       'latitude_merged', 'Ri_01', 'Ri_12', 'N_01', 'N_12',
       'station_elevation', 'landscape_points', 'elevations'],
      dtype='object')

In [112]:
df_toWrite = df_toWrite.rename({'X_merged':'X', 'Y_merged':'Y', 'f_merged':'f', 'fg_merged':'fg', 'd_merged':'d', 'longitude_merged':'longitude', 'latitude_merged':'latitude'}, axis = 1)

df_toWrite.columns

Index(['X', 'Y', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250',
       'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'stod',
       'f', 'fg', 'd', 'longitude', 'latitude', 'Ri_01', 'Ri_12', 'N_01',
       'N_12', 'station_elevation', 'landscape_points', 'elevations'],
      dtype='object')

In [114]:
df_toWrite.to_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-13-3-24-stripped-with-klst.feather')